# Ayudantía 08: Networking - json

## Ejemplo Japonizador

### Autores: [@bmurtagh01 ](https://github.com/bmurtagh01) & [@StroveLight ](https://github.com/StroveLight ) & [@JoacoCoyu](https://github.com/JoacoCoyu)


### Enunciado

El nuevo y revolucionario "Japonizador", es una herramienta que permite que le envíes una palabra cualquiera y te responde con la versión "japonizada" de la misma, es decir, la misma palabra, pero escrita con caracteres y sílabas del idioma japonés.

Además de lo anterior, el "Japonizador" cuenta con un entretenido lado social, pues junto con responder los request de cada cliente, el servidor enviará a todos los usuarios actualmente conectados la última consulta que este recibió, ¡así podrás ver, en tiempo real, qué palabras se están "japonizando" alrededor del mundo!

Para cumplir con lo anterior, cada cliente posee una interfaz gráfica donde podrá ver su última palabra japonizada, junto con la última consulta que ha respondido el servidor.

Al igual que el ejemplo anterior, el objetivo de este programa es poder enviar requests a un servidor y que este las responda a cada cliente, pero además, queremos que el servidor pueda enviar información a todos los clientes sin necesidad de que estos soliciten nada.

#### Archivos incluidos
* ```cliente/```
    * ```cliente.py```: Es el cliente
    * ```interfaz.py```: Es la interfaz
* ```servidor/```
    * ```servidor.py```: Es el servidor
    * ```traductor.py```: Provee la funcion traducir, que japoniza una palabra
    * ```diccionario.json```: Diccionario en formato json que provee la conversion a algunos caracteres japoneses en funcion del sonido

![img/japonizador.jpg](img/japonizador.jpg)

#### Consideraciones
* Antes que todo, te recomendamos ver la explicación del ejercicio propuesto de pickle (el Doggo), ya que ahí se dice mas detalladamente como se envía y recibe usando el protocolo de los 4 bytes

* El codigo se presenta fraccionado (dividido en varias celdas) para facilitar la lectura, pero puedes ver y probar los archivos de solución en la carpeta ```Ejemplo json (Japonizador)/```.

### Primero echémosle un vistazo al ```servidor.py```
#### Primero los import

In [ ]:
import threading
import socket
import json
from traductor import traducir

Cabe mencionar que traducir recibe una palabra y entrega una tupla

#### Server.\_\_init\_\_()

In [ ]:
class Server:

    '''
    Esta es la clase encargada de montar el servidor y realizar las operaciones
    lógicas necesarias para el funcionamiento del sistema
    '''

    def __init__(self):
        print("Inicializando servidor...")
        # Primero se crea un diccionario de sockets para poder guardar
        # a varios clientes de manera simultanea
        self.sockets = dict()

        # Ponemos la dirección donde va a estar situado nuestro servidor
        # El localhost representa "esta computadora" en cualquier red.
        self.host = "localhost"

        # Definimos un puerto al cual el servidor va a estar escuchando
        self.port = 1238

        # Inicializamos el socket principal del servidor.
        self.socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # El método bind enlaza el servidor con el puerto y la dirección.
        # Cualquier cosa que llegue al puerto elegido va a ser 'escuchado' por
        # el servidor
        self.socket_servidor.bind((self.host, self.port))
        print("Dirección y puerto enlazados..")

        # Luego, con el método listen hacemos que el servidor escuche conexiones
        # entrantes. Se le puede pasar como argunento un numero entero, el cual
        # representa el número de conexiones máxima que va a tener en espera de ser aceptadas.
        # En este caso no se puso un argumento, ya que no se necesita una restriccion de esto
        self.socket_servidor.listen()
        print("Servidor escuchando en {}:{}...".format(self.host, self.port))

        # Inicializamos un thread para aceptar conexiones entrantes. Es útil
        # usar un thread para que el programa pueda realizar otras cosas
        # mientras acepta conexiones
        thread = threading.Thread(target=self.accept_connections_thread, daemon=True)
        thread.start()
        print("Servidor aceptando conexiones...")
        
    # (...)

Como se puede observar, el servidor crea un socket, lo enlaza a un host y servidor, y luego se pone al socket a escuchar clientes. Para poder escuchar varios clientes, se implementa un thread que se encargará de ello.

#### Server.accept_connections_thread()
Esta es la función que pusimos como target al thread anterior, se encarga de aceptar conexiones de clientes, y les asigna un thread a cada una, que a su vez escuchará a cada cliente en específico.

In [ ]:
class Server:
    # (...)
    def accept_connections_thread(self):
        '''
        Este método es utilizado en el thread para ir aceptando conexiones de
        manera asíncrona al programa principal
        '''
        # primero se indica un id para guardar cada cliente distinto
        id_ = 1
        while True:

            # El método accept espera (queda esperando) hasta que algún cliente
            # se conecte y luego retorna una tupla con el socket del cliente
            # recién conectado y la dirección de ésto.
            # Nosotros sólo guardamos el socket.
            client_socket, _ = self.socket_servidor.accept()

            # Luego, se guarda el socket
            self.sockets[id_] = client_socket


            print("Servidor conectado a un nuevo cliente...")

            # inicializamos un thread para escuchar a la conexión recién aceptada
            # Por lo que cada cliente conectado va a tener su propio Thread
            # escuchándolo.
            # Pasamos como argumento el socket actual y el id del cliente
            listening_client_thread = threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket, id_),
                daemon=True
            )
            listening_client_thread.start()

            # Finalmente se cambia el valor del id_ para evitar repeticiones
            id_ += 1

#### Server.listen_client_thread()

Esta funcion funciona como un thread que escucha a un cliente específico: Para cada cliente se crea un thread.

Al "escuchar" al cliente, este método se encarga de recibir lo que manda el cliente, y de hacer algo con esa informacion antes de responderle usando ```self.manejar_comando```

La funcion tambien se encarga de deserializar los datos antes de hacer algo con ellos (Su forma de actuar es muy equivalente a la funcion recieve del ejemplo de pickle)

In [ ]:
class Server:
    # (...)
    def listen_client_thread(self, client_socket, id_cliente):
        '''
        Este método va a ser usado múltiples veces en threads pero cada vez con
        sockets de clientes distintos.
        :param client_socket: objeto socket correspondiente a algún cliente
        '''
        while True:
            try:
                # Primero recibimos los 4 bytes del largo
                response_bytes_length = client_socket.recv(4)
                # Los decodificamos
                response_length = int.from_bytes(response_bytes_length,
                                                 byteorder="big")

                # Luego, creamos un bytearray vacío para juntar el mensaje
                response_bytes = bytearray()

                # Recibimos datos hasta que alcancemos la totalidad de los datos
                # indicados en los primeros 4 bytes recibidos.
                while len(response_bytes) < response_length:
                    largo_por_recibir = min(response_length - len(response_bytes), 256)
                    response_bytes += client_socket.recv(largo_por_recibir)

                # Una vez que tenemos todos los bytes, entonces ahí decodificamos
                response = response_bytes.decode()

                # Luego, debemos cargar lo anterior utilizando json
                decoded = json.loads(response)

                # Para evitar hacer muy largo este método, el manejo del mensaje se
                # realizará en otro método
                self.manejar_comando(decoded, client_socket)
            except ConnectionResetError:  # Es decir, si el cliente se desconecta
                del self.sockets[id_cliente]
                break

#### send()
Este método se encarga de enviar la informacion al cliente usando el protocolo de que los primeros 4 bytes indican el tamaño del archivo

In [ ]:
class Server:
    # (...)
    @staticmethod
    def send(mensaje, socket):
        '''
        Este método envía la información al cliente correspondiente al socket.
        :param msg: diccionario del tipo {"mensaje": contenido del mensaje}
        :param socket: socket del cliente al cual se le enviará el mensaje
        :return:
        '''

        # Le hacemos json.dumps y luego lo transformamos a bytes
        msg_json = json.dumps(mensaje)
        msg_bytes = msg_json.encode()

        # Luego tomamos el largo de los bytes y creamos 4 bytes de esto
        msg_length = len(msg_bytes).to_bytes(4, byteorder="big")

        # Finalmente, los enviamos al servidor
        socket.send(msg_length + msg_bytes)

#### Server.sendall()
Este método se encarga de enviar an todos los clientes el último request de algun cliente, para ello, por cada socket hace un send(). Además maneja los casos en los que algún cliente se desconecta

In [ ]:
class Server:
    # (...)
    
    def sendall(self, mensaje):
        # primero se copian las id de los sockets
        id_sockets = list(self.sockets.keys())[:]
        # y luego se iteran sobre ellos
        for id_ in id_sockets:
            try:
                self.send(mensaje, self.sockets[id_])
            except ConnectionResetError:
                del self.sockets[id_]
                print('Error de conexion con cliente')
            except ConnectionAbortedError:
                del self.sockets[id_]
                print('Error de conexion con cliente')
            except IndexError:
                print('Ya se ha eliminado el cliente del diccionario')


#### Server.manejar_comando()
Esta funcion se encarga de manejar un mensaje recibido por el cliente, además de llamar los métodos send y sendall para enviarselos a los otros clientes

In [ ]:
class Server:
    # (...)
    def manejar_comando(self, recibido, socket):
        '''
        Este método toma lo recibido por el cliente correspondiente al socket pasado
        como argumento.
        :param received: diccionario de la forma: {"palabra": Palabra recibida}
        :param client_socket: socket correspondiente al cliente que envió el mensaje
        :return:
        '''

        # Podemos imprimir para verificar que toodo anda bien
        print("Mensaje Recibido: {}".format(recibido))

        palabra = recibido['palabra']
        palabra_fonetica, palabra_traducida = traducir(palabra)

        mensaje = {"propio": True,
                   "original": palabra,
                   "fonetica": palabra_fonetica,
                   "traducida": palabra_traducida}

        # primero le enviamos la respuesta al que pidio la conversion
        self.send(mensaje, socket)

        # despues le actualizamos la ultima consulta a todas los clientes
        mensaje.update({"propio": False})
        self.sendall(mensaje)

### Ahora veamos ```Cliente.py```
Este archivo posee la clase cliente, la cual se conecta a un servidor y usa una interfaz (que está en otro archivo) para mostrar la información. La interfaz la abre solo si se logra conectar con el servidor. La clase Cliente deberá de heredar de QObject para poder comunicarse con la ventana

#### Primero vemos los import

In [ ]:
import threading
import socket
import json
from PyQt5.QtCore import pyqtSignal, QObject  # para poder comunicarse con la ventana
from PyQt5.QtWidgets import QApplication  # Para que al ejecutarse pueda haber ventana
import sys  # Para ejecutarse con ventana apropiadamente

from interfaz import Interfaz  # La ventana

#### Client.\_\_init\_\_()
Primero se declara la señal para comunicarse con la interfaz fuera del \_\_init\_\_, y se hereda de QObject.

Posteriormente se crea un socket, y se hace que este se __conecte__ al servidor. en caso de que no haya servidor, se cancela la operación. 

Cuando se conecta, crea un thread para escuchar al servidor, y abre la interfaz

In [ ]:
class Client(QObject):
    
    # Señal para avisar cuando llegan resultados del servidor
    enviar_a_interfaz = pyqtSignal(dict)

    def __init__(self):

        # Como heredamos de QObject hay que hacer el llamado a super()
        super().__init__()
        print("Inicializando cliente...")

        # Inicializamos el socket principal del cliente.
        # Este corresponde al de una conexión TCP
        self.socket_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Primero definimos la dirección a la cual nos conectaremos.
        # En este caso se trabaja de manera local
        self.host = "localhost"

        # Definimos un Puerto al cual será conectado el cliente
        self.port = 1238

        try:
            # Primero nos conectamos al servidor, pasándole como argumento la tupla
            # (HOST, PORT) al cual nos queremos conectar.
            # Esto tira error si la conexión es privada o si no hay un servidor "escuchando"
            self.socket_cliente.connect((self.host, self.port))
            print("Cliente conectado exitosamente al servidor...")

            # hacemos True un boolean para que escuche
            self.connected = True

            # Luego, creamos un thread para comenzar a escuchar lo que nos envía el servidor
            # Usamos un thread para permitir que el programa realice otras cosas
            # mientras escucha al servidor
            thread = threading.Thread(target=self.listen_thread, daemon=True)
            thread.start()
            print("Escuchando al servidor...")

            # Finalmente, conectamos la señal con un método de la ventana
            self.interfaz = Interfaz()
            self.interfaz.senal_a_backend.connect(self.enviar_al_servidor)
            self.enviar_a_interfaz.connect(self.interfaz.desplegar_resultado)

        except ConnectionRefusedError:
            # Si la conexión es rechazada, entonces se 'cierra' el socket
            print("No se encontró un servidor\nAbortando...")
            self.socket_cliente.close()
            exit()

#### Client.listen_thread()

Es la funcion que se usará como thread para escuchar del servidor. Su forma de ser es muy similar a Server.listen_client_thread()

Esta funcion recibe el mensaje, lo deserializa, y llama una funcion para ver que hacer con la instruccion

In [ ]:
class Client(QObject):
    # (...)
    
    def listen_thread(self):
        '''
        Este método es el usado en el thread y la idea es que reciba lo que
        envía el servidor. Implementa el protocolo de agregar los primeros
        4 bytes, que indican el largo del mensaje
        :return:
        '''

        # Si desean que un usuario pueda desconectarse
        while self.connected:
            # Primero recibimos los 4 bytes del largo
            response_bytes_length = self.socket_cliente.recv(4)
            # Los decodificamos
            response_length = int.from_bytes(response_bytes_length,
                                             byteorder="big")

            # Luego, creamos un bytearray vacío para juntar el mensaje
            response_bytes = bytearray()

            # Recibimos datos hasta que alcancemos la totalidad de los datos
            # indicados en los primeros 4 bytes recibidos.
            while len(response_bytes) < response_length:
                largo_por_recibir = min(response_length - len(response_bytes), 256)
                response_bytes += self.socket_cliente.recv(largo_por_recibir)

            # Una vez que tenemos todos los bytes, entonces ahí decodificamos
            response = response_bytes.decode()

            # Luego, debemos cargar lo anterior utilizando json
            decoded = json.loads(response)

            # Para evitar hacer muy largo este método, el manejo del mensaje se
            # realizará en otro método
            self.manejar_comando(decoded)

#### Client.manejar_comando()

Ya que solo se recibe informacion que se verá en ventana, se mandará la señal a la ventana con la informacion mediante una señal

In [ ]:
class Client(QObject):
    # (...)
    
    def manejar_comando(self, mensaje):
        '''
        :param mensaje: diccionario con la información
        :return:
        '''
        # Podemos imprimir para verificar que toodo anda bien
        print("Mensaje Recibido: {}".format(mensaje))
        
        self.enviar_a_interfaz.emit(mensaje)

#### Client.enviar_al_servidor()
Es una funcion que se activa mediante una señal de la ventana, recibe un string con la palabra y la convierte a un diccionario para después llevar la informacion al servidor

In [ ]:
class Client(QObject):
    # (...)
    def enviar_al_servidor(self, palabra):
        '''
        Este método es el que se gatilla con la señal y manda la información al método send
        :param palabra: string que representa la palabra a japonizar
        :return:
        '''

        # Tomamos la información del evento y la pasamos al formato antes descrito
        data = {"palabra": palabra}

        # Llamamos al método send para enviar la info al servidor
        self.send(data)

#### Client.send()

funcion que se encarga de usar el socket para enviar la informacion al servidor usando el protocolo de enviar los primeros 4 bytes con la informacion del mensaje y luego el mensaje

In [ ]:
class Client(QObject):
    # (...)
    def send(self, msg):
        '''
        :param msg: diccionario con la información
        :return:
        '''

        # Le hacemos json.dumps y luego lo transformamos a bytes
        msg_json = json.dumps(msg)
        msg_bytes = msg_json.encode()

        # Luego tomamos el largo de los bytes y creamos 4 bytes de esto
        msg_length = len(msg_bytes).to_bytes(4, byteorder="big")

        # Finalmente, los enviamos al servidor
        self.socket_cliente.send(msg_length + msg_bytes)